# Game Store Rental Management System

**25COA122 Coursework**

Student ID: F435123

The following files should be uploaded in the next cell:
1. Board_Game_Info.txt
2. Video_Game_Info.txt
3. Rental.txt
4. Game_Feedback.txt
5. Subscription_Info.txt
6. Bookings.txt
7. subscriptionManager.pyc
8. feedbackManager.pyc

These files **MUST** be uploaded

In [4]:
# Upload the .pyc and .txt files.
from google.colab import files
print("Please select the file to upload:")
uploaded = files.upload()

Please select the file to upload:


In [5]:
# Use this to check if the files have been uploaded
!ls

Board_Game_Info.txt  gameRent.py    sample_data
booking.py	     gameReturn.py  Subscription_Info.txt
Booking.txt	     gameSearch.py  subscriptionManager.pyc
database.py	     pruning.py     Video_Game_Info.txt
feedbackManager.pyc  __pycache__
Game_Feedback.txt    Rental.txt


Now run all the following python cells.

When renting a game your username should be 'lbro'

CELL: database

Author: F435123

Date: 29/12/2025

This cell is the “data layer” for the whole program. It’s where all reading and writing to the coursework text files happens, so the rest of the code doesn’t need to deal with file handling.

How to use:

1. Run this cell before any other module.
2. Main functions (and what the parameters mean):
- read_file(filename): reads a pipe “|” file and returns rows as dictionaries.
  - filename: the file to load (e.g., Rental.txt).
- append_to_file(filename, row): adds one new line to the end of a file.
  - filename: file to write to
  - row: the full pipe-delimited line you want to add
- overwrite_file(filename, headers, records): rewrites a file from scratch.
  - filename: file to rewrite
  - headers: column names in the correct order
  - records: list of dictionaries to write back

It also provides helper functions like get_all_games(), add_rental(), update_return_date(), add_booking(), and add_feedback() that other modules call.

In [6]:
%%writefile database.py
# database.py
# Common file-handling functions for the Games Store Management System.
# This module reads and writes the text-based database files.
# Date: 29/12/2025
# Student ID: F435123

# File names (must match uploaded files exactly)
BOARD_GAME_FILE = "Board_Game_Info.txt"
VIDEO_GAME_FILE = "Video_Game_Info.txt"
RENTAL_FILE = "Rental.txt"
BOOKING_FILE = "Booking.txt"
FEEDBACK_FILE = "Game_Feedback.txt"

DELIMITER = "|"


# Read a pipe-delimited file and return a list of dictionaries
# First line is treated as the header row
def read_file(filename):
    records = []

    with open(filename, "r") as file:
        lines = file.readlines()

    # If the file has only a header (or is empty), return no records
    if len(lines) <= 1:
        return records

    headers = lines[0].strip().split(DELIMITER)

    # Convert each row into a dictionary using the headers
    for line in lines[1:]:
        values = line.strip().split(DELIMITER)
        record = dict(zip(headers, values))
        records.append(record)

    return records


# Add one row (string) to the end of a file
def append_to_file(filename, row):
    with open(filename, "a") as file:
        file.write(row + "\n")


# Replace the whole file using a header list and a list of dictionaries
def overwrite_file(filename, headers, records):
    with open(filename, "w") as file:
        file.write(DELIMITER.join(headers) + "\n")

        for record in records:
            line = DELIMITER.join(record.get(h, "") for h in headers)
            file.write(line + "\n")


# Return all board game records from Board_Game_Info.txt
def get_board_games():
    return read_file(BOARD_GAME_FILE)


# Return all video game records from Video_Game_Info.txt
def get_video_games():
    return read_file(VIDEO_GAME_FILE)


# Return all games (board + video)
def get_all_games():
    return get_board_games() + get_video_games()


# Return all rental records from Rental.txt
def get_rentals():
    return read_file(RENTAL_FILE)


# Add a new rental record (ReturnDate is left blank)
def add_rental(game_id, rental_date, customer_id):
    row = f"{game_id}{DELIMITER}{rental_date}{DELIMITER}{DELIMITER}{customer_id}"
    append_to_file(RENTAL_FILE, row)


# Fill in the ReturnDate for the open rental of the given game_id
def update_return_date(game_id, return_date):
    rentals = get_rentals()
    headers = ["GameId", "RentalDate", "ReturnDate", "CustomerId"]

    for rental in rentals:
        if rental.get("GameId", "") == game_id and rental.get("ReturnDate", "") == "":
            rental["ReturnDate"] = return_date
            break

    overwrite_file(RENTAL_FILE, headers, rentals)


# Check if a game is currently rented out (open rental has blank ReturnDate)
def is_game_currently_rented(game_id):
    rentals = get_rentals()

    for rental in rentals:
        if rental.get("GameId", "") == game_id and rental.get("ReturnDate", "") == "":
            return True

    return False


# Return all booking records from Booking.txt
def get_bookings():
    return read_file(BOOKING_FILE)


# Add a new booking record to Booking.txt
def add_booking(user_id, booking_date, time, guests):
    row = f"{user_id}{DELIMITER}{booking_date}{DELIMITER}{time}{DELIMITER}{guests}"
    append_to_file(BOOKING_FILE, row)


# Add a feedback record to Game_Feedback.txt
def add_feedback(game_id, customer_id, rating, comment, date):
    row = (
        f"{game_id}{DELIMITER}{customer_id}{DELIMITER}"
        f"{rating}{DELIMITER}{comment}{DELIMITER}{date}"
    )
    append_to_file(FEEDBACK_FILE, row)


# Return all feedback records from Game_Feedback.txt
def get_feedback():
    return read_file(FEEDBACK_FILE)

Overwriting database.py


CELL: gameSearch

Author: F435123

Date: 02/01/2026

This cell handles searching the inventory. It filters board games, video games, or both, and it also checks if each game is currently “Available” or “Rented” by looking at Rental.txt.

How to use:
1. Run database first.
2. The GUI calls search_games() when you click “Run” under Search Games.

Main functions:
- search_games(game_type="all", title_term="", genre_term="")
  - game_type: “board”, “video”, or “all”
  - title_term: text to look for inside the Name
  - genre_term: text to look for inside the Genre
  - returns: list of matching records (with Type + Availability added)
- format_results(records, limit=50)
  - records: results from search_games
  - limit: max lines to print (mainly for quick testing)

In [7]:
%%writefile gameSearch.py
# gameSearch.py
# Functions for searching games by type (board/video), title, and genre.
# Returns matching game records INCLUDING availability.
# Date: 02/01/2026
# Student ID: F435123

import database


# Convert any input to lowercase text and remove leading/trailing spaces
# This makes searches case-insensitive and consistent
def _normalize(text):
    return str(text).strip().lower()


# Check if a game record is a board game
# Board games have the "NoPlayers" field in the record
def _is_board_game_record(record):
    return "NoPlayers" in record


# Check if a game record is a video game
# Video games have the "Platform" field in the record
def _is_video_game_record(record):
    return "Platform" in record


# Work out if a game is Available or Rented
# This checks Rental.txt using database.is_game_currently_rented()
def _availability_for_game_id(game_id):
    if database.is_game_currently_rented(game_id):
        return "Rented"
    return "Available"


# Search for games
# Returns a list of matching game dictionaries with Availability and Type added
def search_games(game_type="all", title_term="", genre_term=""):
    # Normalize search inputs so comparisons are consistent
    gt = _normalize(game_type)
    title_q = _normalize(title_term)
    genre_q = _normalize(genre_term)

    # Choose which list of games to search depending on game type
    if gt == "board":
        games = database.get_board_games()
    elif gt == "video":
        games = database.get_video_games()
    else:
        games = database.get_all_games()

    results = []

    # Loop through games and check if they match the title and genre filters
    for g in games:
        # Title filter: accept all if title search is blank, otherwise substring match
        name_ok = (title_q == "") or (title_q in _normalize(g.get("Name", "")))

        # Genre filter: accept all if genre search is blank, otherwise substring match
        genre_ok = (genre_q == "") or (genre_q in _normalize(g.get("Genre", "")))

        # If both filters pass, include the game in the results list
        if name_ok and genre_ok:
            # Copy the record so the original database record is not modified
            record = dict(g)

            # Add Availability (Available/Rented) based on the rental file
            record["Availability"] = _availability_for_game_id(record.get("GameId", ""))

            # Add a user-friendly Type label for later display in the GUI
            if _is_board_game_record(record):
                record["Type"] = "Board"
            elif _is_video_game_record(record):
                record["Type"] = "Video"
            else:
                record["Type"] = "Unknown"

            results.append(record)

    return results


# Convert a list of game records into a readable text output
def format_results(records, limit=50):
    # If nothing matched, return a clear message
    if not records:
        return "No matching games found."

    lines = []
    shown = records[:limit]

    # Build a one-line summary for each record
    for r in shown:
        base = (
            f"{r.get('GameId', '')} | {r.get('Type', '')} | {r.get('Name', '')} "
            f"| {r.get('Genre', '')} | {r.get('Availability', '')}"
        )

        # Add platform for video games or number of players for board games
        if r.get("Type") == "Video":
            base += f" | {r.get('Platform', '')}"
        elif r.get("Type") == "Board":
            base += f" | {r.get('NoPlayers', '')} players"

        lines.append(base)

    # If there are more results than the limit, show how many were hidden
    if len(records) > limit:
        lines.append(f"... ({len(records) - limit} more results not shown)")

    return "\n".join(lines)

Overwriting gameSearch.py


CELL: gameRent

Author: F435123

Date: 03/01/2026

This cell deals with renting games. It checks the customer ID, confirms the customer has an active subscription (subscriptionManager.pyc), checks the game exists, checks it’s not already rented, then logs the rental to Rental.txt.

How to use:
1. Run database, then run this cell.
2. The GUI calls rent_game() when you use “Rent Game”.

Main function:
- rent_game(customer_id, game_id, rental_date=None)
  - customer_id: 4-letter ID (e.g., coab)
  - game_id: the ID of the game being rented
  - rental_date: YYYY-MM-DD, or None to use today
  - returns: (True/False, message)

In [8]:
%%writefile gameRent.py
# gameRent.py
# Renting logic: validate customer subscription + game availability, then log rental.
# Date: 03/01/2026
# Student ID: F435123

import datetime
import database
import subscriptionManager

SUBSCRIPTIONS = subscriptionManager.load_subscriptions()

def _is_valid_customer_id(customer_id):
    """Customer IDs must be 4 alphabetic letters (e.g., coab)."""
    return isinstance(customer_id, str) and len(customer_id) == 4 and customer_id.isalpha()


def _today_str():
    return datetime.date.today().isoformat()


def rent_game(customer_id, game_id, rental_date=None):
    # Attempts to rent a game.
    # Returns a (success: boolean, message: string)
    # Validate inputs
    if not _is_valid_customer_id(customer_id):
        return False, "Invalid customer ID. Must be 4 letters (e.g., coab)."

    if not isinstance(game_id, str) or game_id.strip() == "":
        return False, "Invalid game ID."

    customer_id = customer_id.lower().strip()
    game_id = game_id.lower().strip()

    # Check subscription status
    try:
      is_active = subscriptionManager.check_subscription(customer_id, SUBSCRIPTIONS)
    except Exception as e:
      return False, f"Subscription check failed: {e}"


    if not is_active:
        return False, "Customer does not have an active subscription."

    # Check game exists in inventory
    all_games = database.get_all_games()
    game_exists = any(g.get("GameId", "").lower() == game_id for g in all_games)
    if not game_exists:
        return False, "Game ID not found in inventory."

    # Check availability
    if database.is_game_currently_rented(game_id):
        return False, "Game is currently rented out."

    # Use date
    if rental_date is None:
        rental_date = _today_str()

    # Write rental record
    database.add_rental(game_id, rental_date, customer_id)
    return True, f"Rental successful: {game_id} rented to {customer_id} on {rental_date}."

Overwriting gameRent.py


CELL: booking

Author: F435123

Date: 04/01/2026

This cell handles booking a face-to-face session. It checks the customer is subscribed, the date is valid, the time slot is either 2pm or 6pm, guests are 0–3, and the session doesn’t exceed capacity (50 people total per slot).

How to use:
1. Run database, then run this cell.
2. The GUI calls book_session() under “Book Session”.

Main functions:
- book_session(customer_id, booking_date, time_slot, guests)
  - customer_id: 4-letter ID
  - booking_date: YYYY-MM-DD
  - time_slot: “2pm” or “6pm”
  - guests: number 0–3
  - returns: (True/False, message)
- slot_summary(booking_date, time_slot)
  - booking_date: YYYY-MM-DD
  - time_slot: “2pm” or “6pm”
  - returns: info about how full the slot

In [9]:
%%writefile booking.py
# booking.py
# Booking logic for face-to-face sessions:
# - Validate customer ID and active subscription
# - Time slots: 2pm or 6pm
# - Guests: 0-3
# - Capacity: max 50 persons per slot (customer + guests)
# Date: 04/01/26
# Student ID: F435123

import datetime
import database
import subscriptionManager


DELIMITER = "|"
ALLOWED_TIMES = {"2pm", "6pm"}
MAX_GUESTS = 3
MAX_PEOPLE_PER_SLOT = 50

# Load the subscription data once so we don’t re-read the file every time someone books
SUBSCRIPTIONS = subscriptionManager.load_subscriptions()


# Check that the customer ID matches the format:
# exactly 4 letters, no numbers or symbols (e.g., "abcd")
def _is_valid_customer_id(customer_id):
    return isinstance(customer_id, str) and len(customer_id) == 4 and customer_id.isalpha()


# Convert a time input like "2pm", "2 pm", " 2PM " into a consistent format ("2pm")
def _normalize_time(t):
    return str(t).strip().lower().replace(" ", "")


# Get today's date in the same format used in the text files (YYYY-MM-DD)
def _today_str():
    return datetime.date.today().isoformat()


# Clean up text so it can safely be written into a pipe-delimited file
# (Stops users from breaking the file format by typing "|" or new lines)
def _sanitize_field(text):
    if text is None:
        return ""
    s = str(text).replace("\n", " ").replace("\r", " ")
    s = s.replace(DELIMITER, " ")
    return s.strip()


# Convert the user’s date input into a valid ISO date string (YYYY-MM-DD)
def _parse_date(date_str):
    s = str(date_str).strip()
    try:
        d = datetime.date.fromisoformat(s)
        return True, d.isoformat(), ""
    except Exception:
        return False, "", "Invalid date. Use YYYY-MM-DD (e.g., 2025-09-01)."


# Work out how many people are already booked for a particular session slot
# Each booking counts as (1 customer + number of guests)
def _people_booked_for_slot(bookings, booking_date, time_str):
    total = 0

    for b in bookings:
        same_date = str(b.get("BookingDate", "")).strip() == booking_date
        same_time = _normalize_time(b.get("Time", "")) == time_str

        if same_date and same_time:
            try:
                guests = int(b.get("Guests", "0"))
            except Exception:
                guests = 0

            # Add 1 for the customer plus their guests
            total += 1 + guests

    return total


# Main booking function
def book_session(customer_id, booking_date, time_slot, guests):
    # Check customer ID format
    if not _is_valid_customer_id(customer_id):
        return False, "Invalid customer ID. Must be 4 letters (e.g., coab)."

    customer_id = customer_id.strip().lower()

    # Checks if the customer has an active subscription (required by spec)
    try:
        active = subscriptionManager.check_subscription(customer_id, SUBSCRIPTIONS)
    except Exception as e:
        return False, f"Subscription check failed: {e}"

    if not active:
        return False, "Customer does not have an active subscription."

    # Validates the booking date
    ok, iso_date, msg = _parse_date(booking_date)
    if not ok:
        return False, msg

    # Validates the time slot (only 2pm or 6pm allowed)
    time_norm = _normalize_time(time_slot)
    if time_norm not in ALLOWED_TIMES:
        return False, "Invalid time slot. Must be '2pm' or '6pm'."

    # Validates number of guests (0 to 3)
    try:
        guests_int = int(guests)
    except Exception:
        return False, "Guests must be an integer (0 to 3)."

    if guests_int < 0 or guests_int > MAX_GUESTS:
        return False, "Guests must be between 0 and 3."

    # Checks capacity (max 50 people total per session slot)
    bookings = database.get_bookings()
    currently_booked = _people_booked_for_slot(bookings, iso_date, time_norm)

    adding = 1 + guests_int
    if currently_booked + adding > MAX_PEOPLE_PER_SLOT:
        return False, (
            f"Booking denied: capacity exceeded for {iso_date} {time_norm}. "
            f"Currently booked {currently_booked}/50, trying to add {adding}."
        )

    # Saves the booking record into Booking.txt
    # Format matches: UserId|BookingDate|Time|Guests
    database.add_booking(customer_id, iso_date, time_norm, str(guests_int))

    return True, (
        f"Booking successful: {customer_id} booked {iso_date} {time_norm} "
        f"with {guests_int} guest(s)."
    )


# Prompts the store manager for booking details using input()
def prompt_booking_one():
    customer_id = input("Enter customer ID (4 letters, e.g., coab): ").strip()
    booking_date = input(f"Enter booking date (YYYY-MM-DD) [default { _today_str() }]: ").strip()
    if booking_date == "":
        booking_date = _today_str()

    time_slot = input("Enter time slot (2pm or 6pm): ").strip()
    guests = input("Number of guests (0-3): ").strip()

    return book_session(customer_id, booking_date, time_slot, guests)


# Useful for testing capacity rules (getting close to 50 people)
def prompt_booking_multiple():
    print("Enter bookings one per line in format: customer_id,YYYY-MM-DD,2pm,guests")
    print("Example: coab,2025-09-01,2pm,1")
    print("Press Enter on a blank line to finish.\n")

    results = []

    while True:
        line = input("> ").strip()
        if line == "":
            break

        parts = [p.strip() for p in line.split(",")]
        if len(parts) != 4:
            results.append((line, False, "Invalid format. Use: customer_id,YYYY-MM-DD,2pm,guests"))
            continue

        cid, d, t, g = parts
        ok, msg = book_session(cid, d, t, g)
        results.append((line, ok, msg))

    return results


# Helper used by the GUI: shows how full a slot currently is
def slot_summary(booking_date, time_slot):
    ok, iso_date, msg = _parse_date(booking_date)
    if not ok:
        return {"ok": False, "message": msg}

    time_norm = _normalize_time(time_slot)
    if time_norm not in ALLOWED_TIMES:
        return {"ok": False, "message": "Invalid time slot. Must be '2pm' or '6pm'."}

    bookings = database.get_bookings()
    people = _people_booked_for_slot(bookings, iso_date, time_norm)

    return {
        "ok": True,
        "date": iso_date,
        "time": time_norm,
        "people_booked": people,
        "capacity": MAX_PEOPLE_PER_SLOT,
        "spaces_left": MAX_PEOPLE_PER_SLOT - people
    }


Overwriting booking.py


CELL: gameReturn

Author: F435123

Date: 06/01/2026

This cell returns a game that’s currently rented. It finds the “open” rental (ReturnDate is blank), fills in the return date in Rental.txt, and (if chosen) stores feedback. If feedbackManager.pyc is available it uses that; otherwise it falls back to writing to Game_Feedback.txt.

How to use:
1. Run database first.
2. The GUI calls return_game() under “Return Game”.

Main function:
- return_game(game_id, rating=None, comment="", return_date=None)
  - game_id: the game being returned
  - rating: 1–5, or None for no feedback
  - comment: optional text comment
  - return_date: YYYY-MM-DD, or None to use today
  - returns: (True/False, message)

In [10]:
%%writefile gameReturn.py
# gameReturn.py
# Return logic: validate returnable game(s), update Rental.txt, and collect feedback
# Date: 06/01/2026
# Student ID: F435123

import datetime
import database

try:
    import feedbackManager
except Exception:
    feedbackManager = None


DELIMITER = "|"


# Get today's date as YYYY-MM-DD (same format used in the files)
def _today_str():
    return datetime.date.today().isoformat()


# Clean comment text so it won't break the pipe-delimited text file format
def _sanitize_field(text):
    if text is None:
        return ""
    s = str(text)
    s = s.replace("\n", " ").replace("\r", " ")
    s = s.replace(DELIMITER, " ")
    return s.strip()


# Find the rental record for a game that is currently rented
# Returns the open rental dict, or None if the game is not currently rented
def _find_open_rental(game_id):
    game_id = str(game_id).strip().lower()
    rentals = database.get_rentals()

    for r in rentals:
        if str(r.get("GameId", "")).strip().lower() == game_id and str(r.get("ReturnDate", "")) == "":
            return r

    return None

# Returns True if feedback was saved using feedbackManager, otherwise False
def _write_feedback_via_feedback_manager(game_id, customer_id, rating, comment, date_str):
    if feedbackManager is None:
        return False

    candidates = [
        "add_feedback",
        "store_feedback",
        "record_feedback",
        "submit_feedback",
        "save_feedback",
        "insert_feedback",
        "collect_feedback",
    ]

    for name in candidates:
        fn = getattr(feedbackManager, name, None)
        if callable(fn):
            try:
                fn(game_id, customer_id, rating, comment, date_str)
                return True
            except TypeError:
                pass

            try:
                fn(game_id, rating, comment)
                return True
            except TypeError:
                pass

            try:
                fn(game_id, customer_id, rating, comment)
                return True
            except TypeError:
                pass

            try:
                fn(game_id, rating)
                return True
            except TypeError:
                pass
            except Exception:
                # If feedbackManager exists but fails, do not crash the return process
                return False

    return False


# Main return function used by the GUI/menu
def return_game(game_id, rating=None, comment="", return_date=None):
    # Validate game ID input
    if not isinstance(game_id, str) or game_id.strip() == "":
        return False, "Invalid game ID."

    game_id = game_id.strip().lower()

    # Check the game exists in the inventory files
    all_games = database.get_all_games()
    game_exists = any(str(g.get("GameId", "")).strip().lower() == game_id for g in all_games)
    if not game_exists:
        return False, "Game ID not found in inventory."

    # Check the game has an open rental (so it can be returned)
    open_rental = _find_open_rental(game_id)
    if open_rental is None:
        return False, "Game is not currently rented / not returnable."

    # Get the customer who rented the game
    customer_id = str(open_rental.get("CustomerId", "")).strip().lower()

    # Use today's date if no return date is provided
    if return_date is None:
        return_date = _today_str()

    # Update the open rental record by filling in ReturnDate
    database.update_return_date(game_id, return_date)

    # If rating is provided, store feedback (rating 1-5 + optional comment)
    if rating is not None:
        # Check rating is a valid number
        try:
            rating_int = int(rating)
        except Exception:
            return False, "Return recorded, but rating was invalid (must be an integer 1–5)."

        # Check rating range
        if rating_int < 1 or rating_int > 5:
            return False, "Return recorded, but rating must be between 1 and 5."

        safe_comment = _sanitize_field(comment)

        # Try using feedbackManager
        stored = _write_feedback_via_feedback_manager(
            game_id=game_id,
            customer_id=customer_id,
            rating=str(rating_int),
            comment=safe_comment,
            date_str=return_date,
        )

        # If feedbackManager isn't available, save feedback using the database file function
        if not stored:
            database.add_feedback(game_id, customer_id, str(rating_int), safe_comment, return_date)

    return True, f"Return successful: {game_id} returned on {return_date}."


# Prompt-based return for one game
def prompt_return_one():
    game_id = input("Enter Game ID to return (e.g., mon01): ").strip()

    # Only continue if the game is actually rented out
    open_rental = _find_open_rental(game_id)
    if open_rental is None:
        return False, "Game is not currently rented / not returnable."

    # Ask if feedback should be collected
    give_feedback = input("Add feedback now? (y/n): ").strip().lower()
    if give_feedback == "y":
        rating = input("Star rating (1-5): ").strip()
        comment = input("Optional comment (press Enter to skip): ")
        return return_game(game_id, rating=rating, comment=comment)

    return return_game(game_id, rating=None, comment="")


# Prompt-based return for multiple games
# Returns a list of results for each game
def prompt_return_multiple():
    raw = input("Enter Game IDs to return (comma-separated): ").strip()
    if raw == "":
        return [("", False, "No game IDs provided.")]

    game_ids = [g.strip() for g in raw.split(",") if g.strip() != ""]
    results = []

    for gid in game_ids:
        # Skip games that are not currently rented
        open_rental = _find_open_rental(gid)
        if open_rental is None:
            results.append((gid, False, "Not returnable (not currently rented)."))
            continue

        # Ask feedback per game
        give_feedback = input(f"Add feedback for {gid}? (y/n): ").strip().lower()
        if give_feedback == "y":
            rating = input("  Star rating (1-5): ").strip()
            comment = input("  Optional comment: ")
            ok, msg = return_game(gid, rating=rating, comment=comment)
            results.append((gid, ok, msg))
        else:
            ok, msg = return_game(gid, rating=None, comment="")
            results.append((gid, ok, msg))

    return results

Overwriting gameReturn.py


CELL: pruning

Author: F435123

Date: 08/01/2026

This cell looks at rental history and suggests games that might be removed because they are rarely rented. It doesn’t delete anything — it only produces suggestions and graphs to help the manager decide.

How to use:
1. Run database first.
2. The GUI calls suggest_pruning() under “Inventory Pruning”.

Main functions:
- suggest_pruning(min_rentals=2, not_rented_since_days=120, exclude_currently_rented=True)
  - min_rentals: suggest games with rentals <= this
  - not_rented_since_days: suggest games not rented for this many days
  - exclude_currently_rented: avoids suggesting games currently out
  - returns: list of suggested games
- plot_rental_distribution(): shows how rentals are spread across games
- plot_least_rented(top_n=10): shows the bottom N games by rentals
  - top_n: how many games to show
- plot_suggestions(suggestions, top_n=10): graphs the pruning candidates
  - suggestions: list returned from suggest_pruning
  - top_n: how many candidates to show

In [11]:
%%writefile pruning.py
# pruning.py
# Identify games that could be removed based on rental frequency.
# Shows suggestions and graphs (MatPlotLib) to help the manager decide.
# Date: 08/01/2026
# Student ID: F435123

import datetime
import matplotlib.pyplot as plt
import database


# Convert a YYYY-MM-DD string into a date object
# Returns None if the string is empty or invalid
def _parse_iso_date(date_str):
    s = str(date_str).strip()
    if s == "":
        return None
    try:
        return datetime.date.fromisoformat(s)
    except Exception:
        return None


# Decide if a game is a Board game or Video game by checking its fields
def _game_type_from_record(game_record):
    if "NoPlayers" in game_record:
        return "Board"
    if "Platform" in game_record:
        return "Video"
    return "Unknown"


# Make text safe for comparisons (lowercase + strip)
def _safe_lower(x):
    return str(x).strip().lower()


# Read Rental.txt and build stats per GameId
# Tracks total rentals, currently rented, and last rental date
def build_rental_stats():
    rentals = database.get_rentals()
    stats = {}

    for r in rentals:
        gid = _safe_lower(r.get("GameId", ""))
        if gid == "":
            continue

        rental_date = _parse_iso_date(r.get("RentalDate", ""))
        return_date = _parse_iso_date(r.get("ReturnDate", ""))

        if gid not in stats:
            stats[gid] = {
                "total_rentals": 0,
                "currently_rented": False,
                "last_rental_date": None,
                "last_return_date": None,
            }

        stats[gid]["total_rentals"] += 1

        # ReturnDate blank means it is currently rented out
        if str(r.get("ReturnDate", "")).strip() == "":
            stats[gid]["currently_rented"] = True

        # Keep the most recent rental date
        if rental_date is not None:
            cur = stats[gid]["last_rental_date"]
            if cur is None or rental_date > cur:
                stats[gid]["last_rental_date"] = rental_date

        # Keep the most recent return date
        if return_date is not None:
            cur = stats[gid]["last_return_date"]
            if cur is None or return_date > cur:
                stats[gid]["last_return_date"] = return_date

    return stats


# Add the rental stats onto each inventory record
# Returns a list of game records with stats fields added
def annotate_games_with_stats():
    games = database.get_all_games()
    stats = build_rental_stats()

    enriched = []

    for g in games:
        gid = _safe_lower(g.get("GameId", ""))
        s = stats.get(gid, None)

        record = dict(g)
        record["Type"] = _game_type_from_record(g)

        if s is None:
            record["TotalRentals"] = 0
            record["CurrentlyRented"] = False
            record["LastRentalDate"] = ""
        else:
            record["TotalRentals"] = s["total_rentals"]
            record["CurrentlyRented"] = s["currently_rented"]
            record["LastRentalDate"] = s["last_rental_date"].isoformat() if s["last_rental_date"] else ""

        enriched.append(record)

    return enriched


# Suggest games that could be removed
# A game is suggested if it has low rentals and hasn't been rented recently
def suggest_pruning(min_rentals=2, not_rented_since_days=120, exclude_currently_rented=True):
    today = datetime.date.today()
    enriched = annotate_games_with_stats()

    suggestions = []

    for g in enriched:
        total = int(g.get("TotalRentals", 0))
        currently = bool(g.get("CurrentlyRented", False))
        last_rental_str = str(g.get("LastRentalDate", "")).strip()
        last_rental = _parse_iso_date(last_rental_str)

        # Skip games that are currently rented out
        if exclude_currently_rented and currently:
            continue

        # Work out days since last rental (or mark as "Never")
        if last_rental is None:
            days_since = 10**9
        else:
            days_since = (today - last_rental).days

        if total <= min_rentals and days_since >= not_rented_since_days:
            g2 = dict(g)
            g2["DaysSinceLastRental"] = days_since if days_since != 10**9 else "Never"
            suggestions.append(g2)

    # Sort: lowest rentals first, then oldest last rental first
    def sort_key(x):
        total_val = int(x.get("TotalRentals", 0))
        days = x.get("DaysSinceLastRental", 10**9)
        if days == "Never":
            days = 10**9
        return (total_val, -int(days))

    suggestions.sort(key=sort_key)
    return suggestions


# Bar chart of the least rented games (X axis is game names)
def plot_least_rented(top_n=10):
    enriched = annotate_games_with_stats()
    enriched.sort(key=lambda x: int(x.get("TotalRentals", 0)))

    subset = enriched[:max(1, int(top_n))]
    labels = [g.get("Name", g.get("GameId", "")) for g in subset]
    values = [int(g.get("TotalRentals", 0)) for g in subset]

    plt.figure()
    plt.bar(labels, values)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Total rentals")
    plt.title(f"Least Rented Games (Bottom {len(subset)})")
    plt.tight_layout()
    plt.show()


# Histogram showing rental frequency spread across the whole inventory
def plot_rental_distribution():
    enriched = annotate_games_with_stats()
    values = [int(g.get("TotalRentals", 0)) for g in enriched]

    plt.figure()
    plt.hist(values, bins=10)
    plt.xlabel("Total rentals per game")
    plt.ylabel("Number of games")
    plt.title("Distribution of Rental Frequency Across Inventory")
    plt.tight_layout()
    plt.show()


# Bar chart for pruning candidates only (X axis is game names)
def plot_suggestions(suggestions, top_n=10):
    subset = suggestions[:max(1, int(top_n))]
    if not subset:
        print("No suggestions to plot.")
        return

    labels = [g.get("Name", g.get("GameId", "")) for g in subset]
    values = [int(g.get("TotalRentals", 0)) for g in subset]

    plt.figure()
    plt.bar(labels, values)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Total rentals")
    plt.title(f"Pruning Candidates (Top {len(subset)} suggestions)")
    plt.tight_layout()
    plt.show()


# Prompt the manager for pruning settings and show suggestions + graphs
def prompt_pruning():
    try:
        min_rentals = int(input("Min rentals (suggest if <= this number) [default 2]: ").strip() or "2")
    except Exception:
        min_rentals = 2

    try:
        days = int(input("Not rented since days (suggest if >= this) [default 120]: ").strip() or "120")
    except Exception:
        days = 120

    excl = input("Exclude currently rented games? (y/n) [default y]: ").strip().lower() or "y"
    exclude_currently_rented = (excl == "y")

    suggestions = suggest_pruning(
        min_rentals=min_rentals,
        not_rented_since_days=days,
        exclude_currently_rented=exclude_currently_rented
    )

    print("\n--- Pruning Suggestions ---")
    if not suggestions:
        print("No games meet the pruning criteria.")
    else:
        for g in suggestions[:20]:
            print(
                f"{g.get('GameId')} | {g.get('Type')} | {g.get('Name')} | "
                f"Rentals={g.get('TotalRentals')} | LastRental={g.get('LastRentalDate')} | "
                f"DaysSince={g.get('DaysSinceLastRental')}"
            )
        if len(suggestions) > 20:
            print(f"... ({len(suggestions) - 20} more suggestions not shown)")

    print("\n--- Visualisations ---")
    plot_rental_distribution()
    plot_least_rented(top_n=10)
    plot_suggestions(suggestions, top_n=10)

    return suggestions


Overwriting pruning.py


CELL: menu

Author: F435123

Date: 11/01/2026

This is the main user interface. It builds a single-window menu using IPyWidgets, lets the manager choose an action, shows the right inputs for that action, and then runs the correct module function when “Run” is clicked. Search and pruning results are shown as tables.

How to use:
1. Upload all required .txt files + the .pyc files.
2. Run the module cells first (database, gameSearch, gameRent, gameReturn, booking, pruning).
3. Run this cell last to show the GUI.
4. Use “Reload Modules” if you edited a module and want the GUI to pick up changes.
5. Use the username 'lbro' to rent or return games.

Key helper functions in this cell:
- reload_modules(): reloads your modules after edits
- build_search_table(records, limit): turns search results into an HTML table
  - records: results list from gameSearch.search_games
  - limit: max rows to display
- build_pruning_table(records, limit): turns pruning suggestions into an HTML table
  - records: list from pruning.suggest_pruning
  - limit: max rows to display

In [12]:
# GUI Menu (IPyWidgets, single window)
# Student ID: F435123
# Date: 11/01/2026

import importlib
import datetime
import html

import ipywidgets as widgets
from IPython.display import display

import database
import gameSearch
import gameRent
import gameReturn
import booking
import pruning


def reload_modules():
    #Reload all custom modules.
    importlib.reload(database)
    importlib.reload(gameSearch)
    importlib.reload(gameRent)
    importlib.reload(gameReturn)
    importlib.reload(booking)
    importlib.reload(pruning)


def today_iso():
    #Return today's date in YYYY-MM-DD format.
    return datetime.date.today().isoformat()


def show_message(out, text):
    #Print a message inside a widgets
    with out:
        print(text)


def clear_out(out):
    #Clear a widgets.Output area.
    out.clear_output()


def build_search_table(records, limit=50):
    #Build an HTML table for game search results.
    if not records:
        return "<p>No matching games found.</p>"

    shown = records[:limit]

    table_style = "border-collapse:collapse;width:100%;"
    th_style = "border:1px solid #ddd;padding:6px;text-align:left;background:#f5f5f5;color:black;"
    td_style = "border:1px solid #ddd;padding:6px;text-align:left;"

    headers = [
        "GameId",
        "Name",
        "Type",
        "Genre",
        "Availability",
        "Platform/Players",
        "PurchaseDate"
    ]

    rows_html = []
    for r in shown:
        gid = html.escape(str(r.get("GameId", "")))
        name = html.escape(str(r.get("Name", "")))
        gtype = html.escape(str(r.get("Type", "")))
        genre = html.escape(str(r.get("Genre", "")))
        avail = html.escape(str(r.get("Availability", "")))
        purchase = html.escape(str(r.get("PurchaseDate", "")))

        if r.get("Type") == "Video":
            extra = html.escape(str(r.get("Platform", "")))
        elif r.get("Type") == "Board":
            extra = html.escape(str(r.get("NoPlayers", "")))
        else:
            extra = ""

        rows_html.append(
            "<tr>"
            f"<td style='{td_style}'>{gid}</td>"
            f"<td style='{td_style}'><b>{name}</b></td>"
            f"<td style='{td_style}'>{gtype}</td>"
            f"<td style='{td_style}'>{genre}</td>"
            f"<td style='{td_style}'>{avail}</td>"
            f"<td style='{td_style}'>{extra}</td>"
            f"<td style='{td_style}'>{purchase}</td>"
            "</tr>"
        )

    header_html = "".join([f"<th style='{th_style}'>{h}</th>" for h in headers])
    table_html = (
        f"<table style='{table_style}'>"
        f"<tr>{header_html}</tr>"
        + "".join(rows_html) +
        "</table>"
    )

    if len(records) > limit:
        table_html += f"<p><small>Showing {limit} of {len(records)} results.</small></p>"

    return table_html


def build_pruning_table(records, limit=10):
    #Build an HTML table for pruning suggestions.
    if not records:
        return "<p>No games meet the pruning criteria.</p>"

    shown = records[:limit]

    table_style = "border-collapse:collapse;width:100%;background:black;color:white;"
    th_style = "border:1px solid #444;padding:6px;text-align:left;background:black;color:white;"
    td_style = "border:1px solid #444;padding:6px;text-align:left;background:black;color:white;"

    headers = [
        "Name",
        "Type",
        "TotalRentals",
        "LastRentalDate",
        "DaysSinceLastRental",
        "CurrentlyRented",
        "GameId"
    ]

    rows_html = []
    for r in shown:
        name = html.escape(str(r.get("Name", "")))
        gtype = html.escape(str(r.get("Type", "")))
        total = html.escape(str(r.get("TotalRentals", "")))
        last = html.escape(str(r.get("LastRentalDate", "")))
        since = html.escape(str(r.get("DaysSinceLastRental", "")))
        current = html.escape(str(r.get("CurrentlyRented", "")))
        gid = html.escape(str(r.get("GameId", "")))

        rows_html.append(
            "<tr>"
            f"<td style='{td_style}'><b>{name}</b></td>"
            f"<td style='{td_style}'>{gtype}</td>"
            f"<td style='{td_style}'>{total}</td>"
            f"<td style='{td_style}'>{last}</td>"
            f"<td style='{td_style}'>{since}</td>"
            f"<td style='{td_style}'>{current}</td>"
            f"<td style='{td_style}'>{gid}</td>"
            "</tr>"
        )

    header_html = "".join([f"<th style='{th_style}'>{h}</th>" for h in headers])
    table_html = (
        f"<table style='{table_style}'>"
        f"<tr>{header_html}</tr>"
        + "".join(rows_html) +
        "</table>"
    )

    if len(records) > limit:
        table_html += f"<p><small>Showing {limit} of {len(records)} suggestions.</small></p>"

    return table_html


header = widgets.HTML("<h2>Games Store Management System</h2>")

status_out = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="8px"))
main_out = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="8px"))

search_table_out = widgets.HTML(
    value="",
    layout=widgets.Layout(border="1px solid #ccc", padding="8px")
)

reload_btn = widgets.Button(description="Reload Modules", button_style="")
reload_note = widgets.HTML("<small>Use if you edit .py files while the notebook is running.</small>")


def on_reload_clicked(_):
    #Click handler for the Reload Modules button.
    #Reloads custom modules and reports success/failure in the status box
    clear_out(status_out)
    try:
        reload_modules()
        show_message(status_out, "Modules reloaded successfully.")
    except Exception as e:
        show_message(status_out, f"Reload failed: {e}")


reload_btn.on_click(on_reload_clicked)

action_dd = widgets.Dropdown(
    options=[
        "Search Games",
        "Rent Game",
        "Return Game",
        "Book Session",
        "Inventory Pruning"
    ],
    value="Search Games",
    description="Action:",
    layout=widgets.Layout(width="420px")
)

run_btn = widgets.Button(description="Run", button_style="success")
clear_btn = widgets.Button(description="Clear Output", button_style="")


def on_clear_clicked(_):
    """
    Click handler for the Clear Output button.
    Clears the main output area (prints/plots).
    """
    clear_out(main_out)


clear_btn.on_click(on_clear_clicked)

search_type = widgets.Dropdown(
    options=[("All", "all"), ("Board", "board"), ("Video", "video")],
    value="all",
    description="Type:",
    layout=widgets.Layout(width="300px")
)

search_title = widgets.Text(
    value="",
    description="Title:",
    placeholder="e.g., chess, mario",
    layout=widgets.Layout(width="420px")
)

search_genre = widgets.Text(
    value="",
    description="Genre:",
    placeholder="e.g., strategy, sports",
    layout=widgets.Layout(width="420px")
)

search_limit = widgets.BoundedIntText(
    value=50,
    min=1,
    max=500,
    step=1,
    description="Limit:",
    layout=widgets.Layout(width="220px")
)

rent_customer = widgets.Text(
    value="",
    description="Customer ID:",
    placeholder="4 letters (e.g., abcd)",
    layout=widgets.Layout(width="420px")
)

rent_gameid = widgets.Text(
    value="",
    description="Game ID:",
    placeholder="e.g., mon01, gta11",
    layout=widgets.Layout(width="420px")
)

rent_date = widgets.Text(
    value="",
    description="Date:",
    placeholder="YYYY-MM-DD (blank = today)",
    layout=widgets.Layout(width="420px")
)

return_gameid = widgets.Text(
    value="",
    description="Game ID:",
    placeholder="e.g., mon01",
    layout=widgets.Layout(width="420px")
)

return_add_feedback = widgets.Checkbox(
    value=True,
    description="Collect feedback",
    indent=False
)

return_rating = widgets.BoundedIntText(
    value=5,
    min=1,
    max=5,
    step=1,
    description="Rating:",
    layout=widgets.Layout(width="220px")
)

return_comment = widgets.Text(
    value="",
    description="Comment:",
    placeholder="optional",
    layout=widgets.Layout(width="420px")
)

return_date = widgets.Text(
    value="",
    description="Return Date:",
    placeholder="YYYY-MM-DD (blank = today)",
    layout=widgets.Layout(width="420px")
)

book_customer = widgets.Text(
    value="",
    description="Customer ID:",
    placeholder="4 letters (e.g., abcd)",
    layout=widgets.Layout(width="420px")
)

book_date = widgets.Text(
    value="",
    description="Date:",
    placeholder="YYYY-MM-DD (blank = today)",
    layout=widgets.Layout(width="420px")
)

book_time = widgets.Dropdown(
    options=[("2pm-6pm", "2pm"), ("6pm-10pm", "6pm")],
    value="2pm",
    description="Time:",
    layout=widgets.Layout(width="300px")
)

book_guests = widgets.BoundedIntText(
    value=0,
    min=0,
    max=3,
    step=1,
    description="Guests:",
    layout=widgets.Layout(width="220px")
)

book_show_slot = widgets.Button(description="Check Slot Capacity", button_style="")


def on_check_slot(_):
    #Click handler for Check Slot Capacity.
    #Calls booking.slot_summary() and prints a simple capacity summary.
    clear_out(main_out)

    d = book_date.value.strip()
    if d == "":
        d = today_iso()

    t = book_time.value

    try:
        summary = booking.slot_summary(d, t)
        with main_out:
            if not summary.get("ok", False):
                print(summary.get("message", "Unknown error"))
            else:
                print(f"Slot: {summary['date']} {summary['time']}")
                print(f"People booked: {summary['people_booked']}/{summary['capacity']}")
                print(f"Spaces left: {summary['spaces_left']}")
    except Exception as e:
        with main_out:
            print(f"Slot summary failed: {e}")


book_show_slot.on_click(on_check_slot)

prune_min_rentals = widgets.BoundedIntText(
    value=2,
    min=0,
    max=999,
    step=1,
    description="Max rentals:",
    layout=widgets.Layout(width="260px")
)

prune_days = widgets.BoundedIntText(
    value=120,
    min=0,
    max=5000,
    step=10,
    description="Days since:",
    layout=widgets.Layout(width="260px")
)

prune_exclude_current = widgets.Checkbox(
    value=True,
    description="Exclude currently rented",
    indent=False
)

prune_top_n = widgets.BoundedIntText(
    value=10,
    min=1,
    max=50,
    step=1,
    description="Top N:",
    layout=widgets.Layout(width="220px")
)

prune_plot_dist = widgets.Checkbox(value=True, description="Plot distribution", indent=False)
prune_plot_least = widgets.Checkbox(value=True, description="Plot least rented", indent=False)
prune_plot_sugg = widgets.Checkbox(value=True, description="Plot suggestions", indent=False)

panel_box = widgets.VBox()


def update_panel(_=None):
    #Update which input widgets are shown, based on the selected action.
    action = action_dd.value

    if action == "Search Games":
        panel_box.children = [
            widgets.HTML("<b>Search Games</b>"),
            widgets.HBox([search_type, search_limit]),
            search_title,
            search_genre
        ]

    elif action == "Rent Game":
        panel_box.children = [
            widgets.HTML("<b>Rent Game</b>"),
            rent_customer,
            rent_gameid,
            rent_date
        ]

    elif action == "Return Game":
        panel_box.children = [
            widgets.HTML("<b>Return Game</b>"),
            return_gameid,
            return_date,
            return_add_feedback,
            widgets.HBox([return_rating]),
            return_comment
        ]

    elif action == "Book Session":
        panel_box.children = [
            widgets.HTML("<b>Book Session</b>"),
            book_customer,
            book_date,
            widgets.HBox([book_time, book_guests]),
            book_show_slot
        ]

    elif action == "Inventory Pruning":
        panel_box.children = [
            widgets.HTML("<b>Inventory Pruning</b>"),
            widgets.HBox([prune_min_rentals, prune_days]),
            prune_exclude_current,
            widgets.HBox([prune_top_n]),
            widgets.HBox([prune_plot_dist, prune_plot_least, prune_plot_sugg])
        ]


action_dd.observe(update_panel, names="value")
update_panel()


def on_run_clicked(_):
    #Click handler for Run.
    #Runs the selected action and shows the output in the main output area.
    clear_out(main_out)
    search_table_out.value = ""

    action = action_dd.value

    if action == "Search Games":
        gt = search_type.value
        title = search_title.value
        genre = search_genre.value
        limit = int(search_limit.value)

        try:
            results = gameSearch.search_games(game_type=gt, title_term=title, genre_term=genre)
            search_table_out.value = build_search_table(results, limit=limit)
        except Exception as e:
            show_message(main_out, f"Search failed: {e}")

    elif action == "Rent Game":
        cid = rent_customer.value.strip()
        gid = rent_gameid.value.strip()

        d = rent_date.value.strip()
        if d == "":
            d = None

        try:
            ok, msg = gameRent.rent_game(cid, gid, rental_date=d)
            show_message(main_out, msg)
        except Exception as e:
            show_message(main_out, f"Rent failed: {e}")

    elif action == "Return Game":
        gid = return_gameid.value.strip()

        d = return_date.value.strip()
        if d == "":
            d = None

        try:
            if return_add_feedback.value:
                ok, msg = gameReturn.return_game(
                    gid,
                    rating=return_rating.value,
                    comment=return_comment.value,
                    return_date=d
                )
            else:
                ok, msg = gameReturn.return_game(gid, rating=None, comment="", return_date=d)

            show_message(main_out, msg)
        except Exception as e:
            show_message(main_out, f"Return failed: {e}")

    elif action == "Book Session":
        cid = book_customer.value.strip()

        d = book_date.value.strip()
        if d == "":
            d = today_iso()

        t = book_time.value
        g = book_guests.value

        try:
            ok, msg = booking.book_session(cid, d, t, g)
            show_message(main_out, msg)
        except Exception as e:
            show_message(main_out, f"Booking failed: {e}")

    elif action == "Inventory Pruning":
        min_r = int(prune_min_rentals.value)
        days = int(prune_days.value)
        exclude_current = bool(prune_exclude_current.value)
        top_n = int(prune_top_n.value)

        try:
            suggestions = pruning.suggest_pruning(
                min_rentals=min_r,
                not_rented_since_days=days,
                exclude_currently_rented=exclude_current
            )

            search_table_out.value = build_pruning_table(suggestions, limit=top_n)

            with main_out:
                if prune_plot_dist.value:
                    pruning.plot_rental_distribution()
                if prune_plot_least.value:
                    pruning.plot_least_rented(top_n=top_n)
                if prune_plot_sugg.value:
                    pruning.plot_suggestions(suggestions, top_n=top_n)

        except Exception as e:
            show_message(main_out, f"Pruning failed: {e}")


run_btn.on_click(on_run_clicked)

controls_row = widgets.HBox([action_dd, run_btn, clear_btn])
reload_row = widgets.HBox([reload_btn, reload_note])

ui = widgets.VBox([
    header,
    reload_row,
    widgets.HTML("<hr>"),
    controls_row,
    panel_box,
    widgets.HTML("<b>Status</b>"),
    status_out,
    widgets.HTML("<b>Output</b>"),
    search_table_out,
    main_out
])

display(ui)

# **SECURITY CODE ISSUES**

1. I need to be strict with input validation:
If I don’t properly check things like customer IDs, dates, guest numbers, and game IDs, users can enter invalid values that crash the program or mess up the stored data.

2. Hard-coding values can be risky:
Even though this is coursework, hard-coding important values (like file names, “special” IDs, or limits) can cause problems if files get replaced/renamed or if someone relies on fixed values that should really be configurable.

3. No real access control:
My system doesn’t really stop the “wrong” person from doing actions (like renting, returning, or pruning). In a real system I’d need proper login/roles so only authorised users can do manager-only tasks.

4. Error messages can expose too much:
If I print raw exceptions, it can reveal internal details (file names, paths, module names). A safer approach is to show a simple message to the user and keep the detailed error for debugging only.

5. Trusting external modules/files:
I rely on compiled modules like subscriptionManager.pyc and feedbackManager.pyc. If the wrong file is uploaded or the interface changes, the program can behave incorrectly or even run unsafe code. In real life I’d verify the source/version and handle failures more safely.